# Ollama Webpage summarization
 Before executing the code, ensure that the initial setup outlined in the Readme.md file has been completed.

In [8]:
# Import
import requests
from bs4 import BeautifulSoup
from IPython.display import Markdown, display

In [9]:
# Constants

OLLAMA_API = "http://localhost:11434/api/chat"   # 11434 port where ollama runs
HEADERS = {"Content-Type": "application/json"}
MODEL = "llama3.2"  #Model type

### Test: Verifying Connection with Ollama

In [10]:
# Create a messages list. Has the same format  used on OpenAI

messages = [
    {"role": "user", "content": "Describe some of the business applications of Generative AI"}
]

payload = {
        "model": MODEL,
        "messages": messages,
        "stream": False
    }



There is 2 ways of running ollama package:

- Direct HTTP call: you can send HTTP POST requests to the Ollama API
- ollama python package: provides a higher-level interface to interact with the model, abstracting the direct HTTP requests
- Using OpenAI python library to connect to Ollama

This are making web requests locally from my box to my box. And it's connecting to the llama 3.2 model that's being served by llama, and running at localhost:11434


<u> Recommendation:</u>
- If you are primarily using Ollama: Go with ollama Python Package. It’s the cleanest and most idiomatic solution specifically designed for Ollama.
- If you are using both OpenAI and Ollama APIs: Use OpenAI Library (OpenAI) with custom configuration. This provides a unified approach to handling both APIs.
- For advanced or custom needs: Use Direct HTTP Calls (requests.post) for full control and flexibility, especially if you need to customize headers, handle retries, or debug raw API responses.

In [11]:
response = requests.post(OLLAMA_API, json=payload, headers=HEADERS)
print(response.json()['message']['content'])

Generative Artificial Intelligence (AI) has numerous business applications across various industries. Here are some examples:

1. **Content Generation**: Generative AI can create high-quality content such as articles, social media posts, product descriptions, and more. This can help businesses save time and resources, while also improving the consistency and quality of their content.
2. **Design and Visualization**: Generative AI can be used to generate designs, logos, and visualizations for companies. This can include creating unique and innovative design concepts that meet specific brand guidelines.
3. **Customer Service Chatbots**: Generative AI-powered chatbots can provide 24/7 customer support, answering common questions, and helping customers with simple queries.
4. **Marketing Automation**: Generative AI can be used to generate personalized marketing messages, emails, and social media posts based on customer data and behavior.
5. **Product Development**: Generative AI can help p

In [12]:
import ollama

response = ollama.chat(model=MODEL, messages=messages)
print(response['message']['content'])

Generative AI has numerous business applications across various industries. Here are some examples:

1. **Content Generation**: Generative AI can be used to generate high-quality content such as articles, social media posts, product descriptions, and more. This can help businesses save time and money on content creation while maintaining consistency and quality.
2. **Chatbots and Virtual Assistants**: Generative AI-powered chatbots can provide 24/7 customer support, answering frequently asked questions and helping customers with simple inquiries. They can also be used to generate personalized responses to customer queries.
3. **Marketing Automation**: Generative AI can help automate marketing campaigns by generating personalized emails, social media posts, and other content based on customer behavior and preferences.
4. **Product Design**: Generative AI can be used to design new products, such as furniture, electronics, and fashion items. It can also be used to generate 3D models and p

In [13]:
from openai import OpenAI
ollama_via_openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')

response = ollama_via_openai.chat.completions.create(
    model=MODEL,
    messages=messages
)

print(response.choices[0].message.content)

Generative AI has a wide range of business applications across various industries. Here are some examples:

1. **Product Design and Development**: Generative AI can be used to design new products, such as fashion items, furniture, or electronics. It can generate multiple designs based on customer preferences, product requirements, and design constraints.
2. **Marketing and Advertising**: Generative AI can create personalized advertisements, social media posts, and email campaigns based on customer data, behavior, and preferences.
3. **Content Creation**: AI-powered tools like language generators and image generators can create high-quality content, such as blog posts, articles, videos, and social media posts.
4. **Customer Service Chatbots**: Generative AI can be used to build conversational chatbots that can respond to customer inquiries, provide support, and escalate complex issues to human agents.
5. **Data Annotation and Labeling**: Generative AI can speed up data annotation and la

### Project

In [ ]:
#Prompt Setup:

"""We need to create a structure the APIs understand:
#[ {"role": "system", "content": "system message goes here"},
#  {"role": "user", "content": "user message goes here"} ] """


system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."


def user_prompt_for(website): # A function that writes a User Prompt that asks for summaries of websites:
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt


def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]



In [42]:
# Web Scraping 


class Website: # A class to represent a Webpage, analyize the webpage and extract the information

    def __init__(self, url): # Create this Website object from the given url using the BeautifulSoup librarY
        
        self.url = url

        try: #Handling errors
            response = requests.get(url)
            response.raise_for_status()  # Raises HTTPError if the response code is not 200
            soup = BeautifulSoup(response.content, 'html.parser')
            self.title = soup.title.string if soup.title else "No title found"
            for irrelevant in soup.body(["script", "style", "img", "input"]): #Clean some unseful things
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        except requests.exceptions.RequestException as e:
            print(f"Error fetching the URL: {e}")
            return None


In [43]:

#Website Class Testing

try:
    law = Website("https://www.law.com/?slreturn=20250125165349")
    print(law.title)
    print(law.text)
    print(user_prompt_for(law))
    messages_for(law) #Test the summary of the webpage

except Exception as e:
    print(f"Error initializing Website object: {e}")



Law.com | The Premier Source for Global Legal News & Analysis
Newsletters
Events
Sign In
United States
International
MENU
SEARCH
My Law.com
NEW
Sign In
Law.com
Homepage
About Us
Events
Publications
Topics
Insight & Analysis
Surveys & Rankings
Law Firms
Practice Tools & Resources
Communities
Public Notices
All Sections
Membership programs
Law.com Pro
Law.com Pro Mid-Market
Global Leaders in Law
Global Leaders in Law Advisers
Private Client Global Elite
Media brands
Law.com
Law.com Radar
American Lawyer
Corporate Counsel
National Law Journal
Legal Tech News
New York Law Journal
The Legal Intelligencer
The Recorder
Connecticut Law Tribune
Daily Business Review
Daily Report
Delaware Business Court Insider
New Jersey Law Journal
Texas Lawyer
Supreme Court Brief
Litigation Daily
CLOSE
Membership programs
Law.com Pro
Law.com Pro Mid-Market
Global Leaders in Law
Global Leaders in Law Advisers
Private Client Global Elite
Media brands
Law.com
Law.com Radar
American Lawyer
Corporate Counsel
Natio

In [49]:
# Model Execution
import ollama

def summarize(url):  # And now: call the  API, to do the summarization
    website = Website(url) # Web scraping
    try:
        response = ollama.chat(model=MODEL, messages=messages_for(website))  # LLM inference
        return response['message']['content']
    except Exception as e:
        print(f"Error during LLM API call: {e}")
        return None



summarize("https://www.law.com/?slreturn=20250125165349")


'This appears to be the main page of the ALM (American Law Media) website, which provides news, insights, and analysis on the legal industry. Here\'s a breakdown of the content:\n\n**Top Stories**\n\n* "New York-Based Skadden Team Joins White & Case Group in Mexico City for Citigroup Demerger"\n* "No Two Wildfires Alike: Lawyers Take Different Legal Strategies in California"\n* "Poop-Themed Dog Toy OK as Parody, but Still Tarnished Jack Daniel’s Brand, Court Says"\n* "Meet the New President of NY\'s Association of Trial Court Jurists"\n* "Lawyers\' Phones Are Ringing: What Should Employers Do If ICE Raids Their Business?"\n\n**Radar**\n\nThis section highlights trending stories and news in the legal industry. The top 5 trending stories are listed, with brief summaries.\n\n**Your Compass Points**\n\nThis section features data-driven reporting using ALM\'s proprietary resources. It includes a link to the Legal Compass platform, which provides insights on law firms, clients, and industrie

In [48]:

def display_summary(url):# A function to display this nicely in the Jupyter output, using markdown
    summary = summarize(url)
    display(Markdown(summary))

display_summary("https://www.law.com/?slreturn=20250125165349")

This appears to be a sample newsletter or publication from ALM (American Law Institute), a leading provider of business information and intelligence for the legal industry. The content includes news articles, summaries of recent court decisions, and updates on industry trends and events.

Some notable sections include:

* "Data-driven reporting using ALM's proprietary resources" - a mention of Legal Compass, a tool that provides data-driven insights into the performance of law firms.
* "Events and Webcasts" - a section highlighting upcoming events and webcasts, including a discussion on managing partner strategy for 2025.
* "Trending Stories" - a curated list of recent news articles and summaries from ALM publications.
* "Resources" - a section offering links to additional resources and products related to AI in law, e-discovery, and other topics.

The newsletter also includes some general industry information, such as the announcement that Davis Wright Tremaine has turned to Gen AI to teach its associates legal writing.